In [1]:
import DMCLP_Instance_Reader as reader
import DMCLP_Instance_Generator as generator
import DMCLP_Gurobi as grb
import DMCLP_LDA as lda
import DMCLP_CA as ca
import DMCLP_utils as utils
import numpy as np
import pandas as pd
import os
import warnings
warnings.simplefilter('ignore')

In [3]:
args_common = {'Q':{'x':[0,1500],'y':[0,1500],'h':[50,500]},
               'F':48.0855,
               'eta':2.5,
               'alpha':4.88,
               'beta':0.43,
               'B':0.1-21,
              }

args_common['L_min'] = utils.get_L_min(args_common['Q'],args_common['F'],args_common['B'],args_common['eta'])
args_common['bigM'] = utils.get_bigM(args_common['Q'],args_common['F'],args_common['B'],args_common['eta'])

                    
args_lda = {'DC_iter_max':100,
            'DC_gradient_step_size':50,
            'DC_epsilon_gradient':0.1,
            'DC_number_of_replications':1,
            'LDA_halving_iter_max':10,
            'LDA_halving_tolerance':0.01,
            'LDA_epsilon_acc':0.01,
            'LDA_iter_max':1,
            'LDA_CPU_max':14400,
            'verbose':False,
           }

args_grb = {'time_limit':30,
            'show_console':False,
            'compute_IIS':False,
            'write_file':True,
            'write_solution':False,
           }

args_ca = {'w_bar':0.5,
           'delta_w':0.2,
           'd_bar':105,
           'delta_d':0.2,
           'delta_t':0.2,
           'CA_grid_division':10,
           'CA_h_division':10,
           'CA_P':0.9,
           'CA_rho':20,
           'CA_varrho':0.5,
           'CA_K':20,
           'CA_l':20,
           'CA_number_of_replications':10
          }

In [4]:
# define the path to save txt files
basepath = 'C:\\Users\Abc\\Google Drive\\Academic\\07.Codes\\Python Codes\\01_Aerial_Wireless_Communication\\02_INFORMS_JOC\\Publication\\3DMCLP\\Instances\\'

# create a list of trends for w and d
trends = ['inc','dec','random']

# set a number for the number of replications for an instance
number_of_replications = 10

# set values for s
number_of_grids = [25,50,100]

# set values for T
number_of_periods = [10]

# solve instances for each option in the above sets
for nbGrid in number_of_grids:
    for nbPeriod in number_of_periods:
        for trend_w in trends:
            for trend_d in trends:
                for rep_no in range(number_of_replications):
                    
                    instance = 'Result_{}_{}_{}_{}_{}'.format(nbGrid,nbPeriod,trend_w,trend_d,rep_no)
                    
                    # generate instance
                    I = range(nbGrid)
                    T = range(nbPeriod)
                    upsilon_w = utils.upsilon(trend=trend_w)
                    upsilon_d = utils.upsilon(trend=trend_d)
                    Y,W,D,p = generator.generate(args_common['Q'],upsilon_w,upsilon_d,nbGrid=nbGrid,nbPeriod=nbPeriod,seed=rep_no,
                                                w_bar=0.5,d_bar=105,delta_w=0.2,delta_d=0.2,delta_t=0.2,delta_p=0.2)
                    
                    # save instance to a txt file
                    with open('{}\\DMCLP_{}_{}_{}_{}_{}.txt'.format(basepath,nbGrid,nbPeriod,trend_w,trend_d,rep_no),'w') as f:
                        f.write(utils.generate_save_file(Y,W,D,p))
                        f.close()

                    # solve instance with CA
                    args_for_ca = utils.merge_two_dicts(args_ca,args_common)
                    X_ca,cpu_ca = ca.solve(I,T,Y,W,D,p,upsilon_w,upsilon_d,args_for_ca)

                    # solve instance with LDA
                    args_for_lda = utils.merge_two_dicts(args_lda,args_common)
                    X_lda,cpu_lda = lda.solve(I,T,Y,W,D,p,args_for_lda)

                    # solve instance with Gurobi
                    args_for_gurobi = utils.merge_two_dicts(args_grb,args_common)
                    X_grb,cpu_grb = grb.solve(I,T,Y,W,D,p,args_for_gurobi)

                    # write solutions to a txt file
                    utils.write_result(basepath,instance,X_ca,X_lda,X_grb)